In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df['Exited'].value_counts(), df['Geography'].value_counts(), df['Gender'].value_counts()

(Exited
 0    7963
 1    2037
 Name: count, dtype: int64,
 Geography
 France     5014
 Germany    2509
 Spain      2477
 Name: count, dtype: int64,
 Gender
 Male      5457
 Female    4543
 Name: count, dtype: int64)

In [6]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [7]:
df.shape

(10000, 11)

In [8]:
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)

In [9]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [12]:
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [13]:
with open('scaler.pkl','wb') as file:
  pickle.dump(scaler,file)

In [14]:
class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.fc1 = nn.Linear(11, 6)
        self.fc2 = nn.Linear(6, 6)
        self.fc3 = nn.Linear(6, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [15]:
# model = nn.Sequential(
#     nn.Linear(11, 6),
#     nn.ReLU(),
#     nn.Linear(6, 6),
#     nn.ReLU(),
#     nn.Linear(6, 1),
#     nn.Sigmoid()
# )

In [16]:
# from collections import OrderedDict

# model = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(11, 6)),
#     ('relu1', nn.ReLU()),
#     ('fc2', nn.Linear(6, 6)),
#     ('relu2', nn.ReLU()),
#     ('fc3', nn.Linear(6, 1)),
#     ('sigmoid', nn.Sigmoid())
# ]))

In [17]:
model = ChurnModel()

In [18]:
from torchsummary import summary

summary(model, input_size=(11,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 6]              72
            Linear-2                    [-1, 6]              42
            Linear-3                    [-1, 1]               7
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [19]:
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [20]:
EPOCH = 100

for epoch in range(EPOCH):
    inputs = torch.from_numpy(X_train.astype(np.float32))
    target = torch.from_numpy(y_train.values.astype(np.float32)).view(-1,1)

    optimizer.zero_grad()

    output = model(inputs)
    losses = loss(output, target)

    losses.backward()

    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch} Loss: {losses.item():.5f}')

Epoch 0 Loss: 0.78819
Epoch 10 Loss: 0.62488
Epoch 20 Loss: 0.48870
Epoch 30 Loss: 0.46692
Epoch 40 Loss: 0.44312
Epoch 50 Loss: 0.43636
Epoch 60 Loss: 0.43121
Epoch 70 Loss: 0.42809
Epoch 80 Loss: 0.42627
Epoch 90 Loss: 0.42480


In [21]:
with torch.no_grad():
    y_pred = model(torch.from_numpy(X_test.values.astype(np.float32))).numpy()
    y_pred = y_pred.round()

    accuracy = torch.from_numpy(y_pred).eq(torch.from_numpy(y_test.values.astype(np.float32)).view(-1,1)).sum().item() / float(y_test.shape[0])
    print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.80


In [22]:
torch.save(model.state_dict(), 'ChurnModel.pth')

In [23]:
# in h5 format
torch.save(model, 'ChurnModel.h5')

In [24]:
load_model = torch.load('ChurnModel.h5')
load_model.eval()

/var/folders/n_/tb_qk64d4q740jlrg8457tvc0000gn/T/ipykernel_42431/2161700812.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model = torch.load('ChurnModel.h5')


ChurnModel(
  (fc1): Linear(in_features=11, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=6, bias=True)
  (fc3): Linear(in_features=6, out_features=1, bias=True)
)

In [25]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(f"Using {device} device")

Using mps device
